In [122]:
import glob
from PIL import Image
from PIL.Image import Resampling
import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

In [123]:
image_files = glob.glob('../annotated/*.jpg')

In [124]:
def BFS(mask):
    visited = np.zeros_like(mask, dtype=np.int32)

    max_size = 0

    for iy, ix in np.ndindex(mask.shape):
        if mask[iy, ix] == 0 or (visited[iy, ix] == -1 and mask[iy, ix] == 255):
            continue
        else:
            queue = [(iy, ix)]
            size = 0
            while len(queue) > 0:
                y, x = queue.pop()
                visited[y, x] = -1
                size += 1
                for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    if 0 <= y + dy < mask.shape[0] and 0 <= x + dx < mask.shape[1]:
                        if mask[y + dy, x + dx] == 255 and visited[y + dy, x + dx] == 0:
                            queue.append((y + dy, x + dx))

            visited[iy, ix] = size
            max_size = max(max_size, size)

    for iy, ix in np.ndindex(visited.shape):
        if 0 < visited[iy, ix] < max_size and mask[iy, ix] == 255:
            queue = [(iy, ix)]
            while len(queue) > 0:
                y, x = queue.pop()
                mask[y, x] = 0
                for dy, dx in [(0, 1), (0, -1), (1, 0), (-1, 0)]:
                    if 0 <= y + dy < mask.shape[0] and 0 <= x + dx < mask.shape[1]:
                        if mask[y + dy, x + dx] == 255:
                            queue.append((y + dy, x + dx))

    return mask

def display_masks(mask1, mask2):
    mask1_d = np.zeros((*mask1.shape, 3), dtype=np.uint8)
    mask2_d = np.zeros((*mask2.shape, 3), dtype=np.uint8)

    mask1_d[mask1 == 255] = [255, 0, 0]
    mask2_d[mask2 == 255] = [0, 255, 0]

    plt.figure(figsize=(10, 10))
    plt.imshow(mask1_d, alpha=0.5)
    plt.imshow(mask2_d, alpha=0.3)
    plt.colorbar()

In [126]:
for image_file in tqdm(image_files):
    mask_image = Image.open(image_file)
    mask = np.asarray(mask_image)
    mask_fixed = np.zeros_like(mask)
    
    # reject any values that are not 0 or 255
    for iy, ix in np.ndindex(mask.shape):
        if mask[iy, ix] > 127:
            mask_fixed[iy, ix] = 255

    mask_fixed = BFS(mask_fixed)
    display_masks(mask, mask_fixed)
    
    mask_fixed = Image.fromarray(mask_fixed)
    mask_fixed.save("./annotated/" + image_file[image_file.rindex("\\")+1:image_file.rindex(".")] + ".png", "PNG")


100%|██████████| 36/36 [02:38<00:00,  4.40s/it]
